***GENERATED CODE FOR daytolastdecisionclean PIPELINE***
***DON'T EDIT THIS CODE***

**CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM**

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


**TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA**

In [ ]:
import json
from pyspark.sql.functions import round
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DiagnosisCodeID1', 'threshold': 40398.9558275578, 'transformation_label': 'Binarizer'}], 'feature': 'DiagnosisCodeID1', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
                                              'count': '264622', 'mean': '40398.96', 'stddev': '25533.94', 'min': '13437.0', 'max': '94919.0', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}]}, {'feature_label': 'DiagnosisCodeID1', 'threshold': 40398.9558275578, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('DiagnosisCodeID1')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


**AUTOML FUNCTIONS**

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = df.drop(label, axis=1).values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': sparkDF.columns}
    return data


**READING DATAFRAME**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

%run daytolastdecisioncleanHooks.ipynb
try:
	sourcePreExecutionHook()

	daytolastdecisioncleandbfs = DBFSConnector.fetch(spark, "{'url': '/Whitespace/whitespace_days_to_first_last_decision_cleaned.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")

	sourcePostExecutionHook(daytolastdecisioncleandbfs)

except Exception as ex: 
	logging.error(ex)


**TRANSFORMING DATAFRAME**

In [ ]:
%run daytolastdecisioncleanHooks.ipynb
try:
	transformationPreExecutionHook()

	daytolastdecisioncleanautofe = TransformationMain.run(daytolastdecisioncleandbfs,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "_c0", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "264622", "mean": "264400.8", "stddev": "152928.86", "min": "4", "max": "530366", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "first_decision", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "264622", "mean": "15.48", "stddev": "9.55", "min": "1.0", "max": "197.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "last_decision", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "264622", "mean": "28.82", "stddev": "27.18", "min": "1.0", "max": "205.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CPTCodeID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "264622", "mean": "1819.1", "stddev": "1216.09", "min": "1", "max": "4916", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FacilityID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "264622", "mean": "102.33", "stddev": "48.41", "min": "0", "max": "179", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ProviderID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "264622", "mean": "134.24", "stddev": "74.43", "min": "0", "max": "263", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PayerId", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "264622", "mean": "142.39", "stddev": "53.88", "min": "0", "max": "251", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DepartmentID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "264622", "mean": "19.68", "stddev": "11.57", "min": "0", "max": "50", "missing": "0"}}, {"transformationsData": [{"feature_label": "DiagnosisCodeID1", "threshold": 40398.9558275578, "transformation_label": "Binarizer"}], "feature": "DiagnosisCodeID1", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "264622", "mean": "40398.96", "stddev": "25533.94", "min": "13437.0", "max": "94919.0", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LoadYear", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "264622", "mean": "2020.0", "stddev": "0.0", "min": "2020", "max": "2020", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LoadMonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "264622", "mean": "7.0", "stddev": "0.0", "min": "7", "max": "7", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LoadWeek", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "264622", "mean": "28.05", "stddev": "0.34", "min": "28", "max": "31", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LoadDay", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "264622", "mean": "9.38", "stddev": "2.35", "min": "9", "max": "29", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LoadDayofweek", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "264622", "mean": "3.0", "stddev": "0.38", "min": "0", "max": "6", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LoadDayofyear", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "264622", "mean": "191.38", "stddev": "2.35", "min": "191", "max": "211", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DateOfServiceYear", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "264622", "mean": "2019.97", "stddev": "0.18", "min": "2017", "max": "2020", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DateOfServiceMonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "264622", "mean": "3.05", "stddev": "2.12", "min": "1", "max": "12", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DateOfServiceWeek", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "264622", "mean": "11.4", "stddev": "8.34", "min": "1", "max": "52", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DateOfServiceDay", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "264622", "mean": "15.68", "stddev": "8.61", "min": "1", "max": "31", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DateOfServiceDayofweek", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "264622", "mean": "1.94", "stddev": "1.43", "min": "0", "max": "6", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DateOfServiceDayofyear", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "264622", "mean": "77.84", "stddev": "65.42", "min": "1", "max": "365", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DateOfEntryYear", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "264622", "mean": "2020.0", "stddev": "0.0", "min": "2020", "max": "2020", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DateOfEntryMonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "264622", "mean": "2.81", "stddev": "1.43", "min": "1", "max": "5", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DateOfEntryWeek", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "264622", "mean": "10.97", "stddev": "6.28", "min": "1", "max": "22", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DateOfEntryDay", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "264622", "mean": "16.06", "stddev": "8.59", "min": "1", "max": "31", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DateOfEntryDayofweek", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "264622", "mean": "2.08", "stddev": "1.52", "min": "0", "max": "6", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DateOfEntryDayofyear", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "264622", "mean": "70.9", "stddev": "43.95", "min": "1", "max": "152", "missing": "0"}}, {"feature": "DiagnosisCodeID1_binarizer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "264622", "mean": "0.29", "stddev": "0.45", "min": "0.0", "max": "1.0", "missing": "0"}}]}))

	transformationPostExecutionHook(daytolastdecisioncleanautofe)

except Exception as ex: 
	logging.error(ex)


**TRAIN MODEL**

In [ ]:
%run daytolastdecisioncleanHooks.ipynb
try:
	mlPreExecutionHook()

	dataAutoML=functionRegression(daytolastdecisioncleanautofe, ["_c0", "first_decision", "CPTCodeID", "FacilityID", "ProviderID", "PayerId", "DepartmentID", "LoadYear", "LoadMonth", "LoadWeek", "LoadDay", "LoadDayofweek", "LoadDayofyear", "DateOfServiceYear", "DateOfServiceMonth", "DateOfServiceWeek", "DateOfServiceDay", "DateOfServiceDayofweek", "DateOfServiceDayofyear", "DateOfEntryYear", "DateOfEntryMonth", "DateOfEntryWeek", "DateOfEntryDay", "DateOfEntryDayofweek", "DateOfEntryDayofyear", "DiagnosisCodeID1_binarizer"], "last_decision")

	mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


**PREDICT ON TRAINED MODEL**

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

